In [1]:
from dask_kubernetes import KubeCluster, make_pod_spec
from dask.distributed import Client
import dask.array as da

In [2]:
scheduler_spec = make_pod_spec(
    image='ghcr.io/dask/dask:latest',
    memory_limit='16G',
    memory_request='16G',
    cpu_limit=4,
    cpu_request=4
)

In [3]:
worker_spec = make_pod_spec(
    image='ghcr.io/dask/dask:latest',
    memory_limit='4G',
    memory_request='4G',
    cpu_limit=1,
    cpu_request=1
)

Open a new terminal in this Jupyter Lab to automatically run `~/keys/gcloud-auth.sh` at startup. Using the terminal,

* `bash` activate the bash shell.

In [4]:
cluster = KubeCluster(
    scheduler_pod_template=scheduler_spec,
    pod_template=worker_spec,
    namespace='dask' # This should match the namespace created by gcloud-auth.sh with the environment variable from compose.yml.
)

Creating scheduler pod on cluster. This may take some time.


Using the Jupyter Lab terminal,

* `kubectl get all` Check the name of the scheduler pod.

* `kubectl wait --for=condition=Ready pod/<scheduler-pod-name> --timeout=600s` Wait until the scheduler pod is ready.

In [5]:
cluster

Dashboard: http://localhost:60110/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.8.0.131:8786,Workers: 0
Dashboard: http://10.8.0.131:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


Using the WSL terminal (instead of the terminal on this Jupyter Lab),

* `kubectl port-forward service/<service-name> 8700:8787 -n dask` Forward the port from WSL to the dask service on the kubernetes cluster. Copy the service name under the "KubeCluster" in the cell output above.

* Go to `localhost:8700` in the web browser on the local machine to access Dask dashboards.

In [6]:
cluster.scale(10)

In [7]:
with Client(cluster) as client:
    array = da.ones((10000, 10000, 10000))
    print(array.mean().compute())    

/usr/local/lib/python3.10/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| lz4     | 4.0.1          | 4.0.0          | 4.0.0          |
| numpy   | 1.23.1         | 1.23.0         | 1.23.0         |
| python  | 3.10.5.final.0 | 3.8.13.final.0 | 3.8.13.final.0 |
| toolz   | 0.12.0         | 0.11.2         | 0.11.2         |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


1.0


In [8]:
cluster.close()

Using the WSL terminal,

* Press `ctrl` + `c` to stop port-forwarding to the service using `kubectl`.